In [2]:
import datetime

# Класс для банковского счета
class Account:
    # Конструктор класса
    def __init__(self, account_holder, balance=0):
        # Проверяем, что баланс не отрицательный
        if balance < 0:
            print("Ошибка: начальный баланс не может быть отрицательным")
            self._balance = 0  # устанавливаем 0 если баланс отрицательный
        else:
            self._balance = balance
        
        self.holder = account_holder  # имя владельца
        self.operations_history = []  # список для истории операций
        
        # Добавляем первую запись - открытие счета
        first_record = {
            'type': 'open_account',  # тип операции
            'amount': balance,       # сумма
            'timestamp': datetime.datetime.now(),  # дата и время
            'balance_after': self._balance,  # баланс после операции
            'status': 'success',  # статус
            'description': f'Открытие счета. Баланс: {self._balance}'  # описание
        }
        self.operations_history.append(first_record)
    
    # Метод для пополнения счета
    def deposit(self, amount):
        # Проверяем, что сумма положительная
        if amount <= 0:
            # Создаем запись о неудачной операции
            bad_record = {
                'type': 'deposit',
                'amount': amount,
                'timestamp': datetime.datetime.now(),
                'balance_after': self._balance,
                'status': 'fail',
                'description': f'Ошибка: сумма {amount} должна быть больше 0'
            }
            self.operations_history.append(bad_record)
            return False  # возвращаем False если ошибка
        
        # Увеличиваем баланс
        old_balance = self._balance
        self._balance += amount
        
        # Создаем запись об успешной операции
        good_record = {
            'type': 'deposit',
            'amount': amount,
            'timestamp': datetime.datetime.now(),
            'balance_after': self._balance,
            'status': 'success',
            'description': f'Пополнение на {amount}. Баланс был {old_balance}, стал {self._balance}'
        }
        self.operations_history.append(good_record)
        return True  # возвращаем True если успех
    
    # Метод для снятия денег
    def withdraw(self, amount):
        # Проверяем, что сумма положительная
        if amount <= 0:
            # Запись о неудачной операции
            bad_record = {
                'type': 'withdraw',
                'amount': amount,
                'timestamp': datetime.datetime.now(),
                'balance_after': self._balance,
                'status': 'fail',
                'description': f'Ошибка: сумма {amount} должна быть больше 0'
            }
            self.operations_history.append(bad_record)
            return False
        
        # Проверяем, хватает ли денег
        if amount > self._balance:
            # Запись о неудачной операции - недостаточно средств
            bad_record = {
                'type': 'withdraw',
                'amount': amount,
                'timestamp': datetime.datetime.now(),
                'balance_after': self._balance,
                'status': 'fail',
                'description': f'Ошибка: недостаточно средств. Хотите снять {amount}, а на счете только {self._balance}'
            }
            self.operations_history.append(bad_record)
            return False
        
        # Уменьшаем баланс
        old_balance = self._balance
        self._balance -= amount
        
        # Запись об успешной операции
        good_record = {
            'type': 'withdraw',
            'amount': amount,
            'timestamp': datetime.datetime.now(),
            'balance_after': self._balance,
            'status': 'success',
            'description': f'Снятие {amount}. Баланс был {old_balance}, стал {self._balance}'
        }
        self.operations_history.append(good_record)
        return True
    
    # Метод для получения баланса
    def get_balance(self):
        return self._balance
    
    # Метод для получения истории операций
    def get_history(self):
        # Просто возвращаем весь список операций
        return self.operations_history
    
    # Метод для красивого вывода информации о счете
    def show_info(self):
        print(f"Владелец счета: {self.holder}")
        print(f"Текущий баланс: {self._balance}")
        print(f"Всего операций: {len(self.operations_history)}")


# Класс для кредитного счета (наследуется от Account)
class CreditAccount(Account):
    # Конструктор с дополнительным параметром credit_limit
    def __init__(self, account_holder, credit_limit, balance=0):
        # Вызываем конструктор родительского класса
        super().__init__(account_holder, balance)
        
        # Сохраняем кредитный лимит
        self.credit_limit = credit_limit
        
        # Обновляем первую запись в истории
        self.operations_history[0] = {
            'type': 'open_account',
            'amount': balance,
            'timestamp': datetime.datetime.now(),
            'balance_after': self._balance,
            'status': 'success',
            'description': f'Открытие КРЕДИТНОГО счета. Баланс: {self._balance}, Кредитный лимит: {self.credit_limit}',
            'credit_limit': self.credit_limit  # добавляем информацию о кредитном лимите
        }
    
    # Переопределяем метод снятия денег для учета кредитного лимита
    def withdraw(self, amount):
        # Проверяем, что сумма положительная
        if amount <= 0:
            bad_record = {
                'type': 'withdraw',
                'amount': amount,
                'timestamp': datetime.datetime.now(),
                'balance_after': self._balance,
                'status': 'fail',
                'description': f'Ошибка: сумма {amount} должна быть больше 0',
                'credit_used': False,
                'available_credit': self._balance + self.credit_limit
            }
            self.operations_history.append(bad_record)
            return False
        
        # Проверяем с учетом кредитного лимита
        # Можно снять: текущий баланс + кредитный лимит
        max_withdraw = self._balance + self.credit_limit
        
        if amount > max_withdraw:
            bad_record = {
                'type': 'withdraw',
                'amount': amount,
                'timestamp': datetime.datetime.now(),
                'balance_after': self._balance,
                'status': 'fail',
                'description': f'Ошибка: недостаточно средств. Хотите снять {amount}, а доступно {max_withdraw} (с учетом кредитного лимита {self.credit_limit})',
                'credit_used': False,
                'available_credit': max_withdraw
            }
            self.operations_history.append(bad_record)
            return False
        
        # Определяем, используем ли мы кредитные средства
        credit_used = False
        if amount > self._balance:
            credit_used = True
        
        # Сохраняем старый баланс
        old_balance = self._balance
        
        # Вычитаем сумму
        self._balance -= amount
        
        # Запись об успешной операции
        good_record = {
            'type': 'withdraw',
            'amount': amount,
            'timestamp': datetime.datetime.now(),
            'balance_after': self._balance,
            'status': 'success',
            'description': f'Снятие {amount}. Баланс был {old_balance}, стал {self._balance}',
            'credit_used': credit_used,
            'available_credit': self._balance + self.credit_limit
        }
        
        # Добавляем дополнительное описание если использовали кредит
        if credit_used:
            credit_amount = amount - old_balance
            good_record['description'] += f'. Использовано кредитных средств: {credit_amount}'
        
        self.operations_history.append(good_record)
        return True
    
    # Метод для получения доступного кредита
    def get_available_credit(self):
        # Доступный кредит = текущий баланс + кредитный лимит
        # Но если баланс положительный, то просто кредитный лимит
        if self._balance >= 0:
            return self.credit_limit
        else:
            # Если баланс отрицательный, то часть кредита уже использована
            return self._balance + self.credit_limit
    
    # Переопределяем метод пополнения для учета погашения кредита
    def deposit(self, amount):
        # Проверяем, что сумма положительная
        if amount <= 0:
            bad_record = {
                'type': 'deposit',
                'amount': amount,
                'timestamp': datetime.datetime.now(),
                'balance_after': self._balance,
                'status': 'fail',
                'description': f'Ошибка: сумма {amount} должна быть больше 0',
                'credit_used': False,
                'available_credit': self._balance + self.credit_limit
            }
            self.operations_history.append(bad_record)
            return False
        
        # Сохраняем старый баланс
        old_balance = self._balance
        
        # Увеличиваем баланс
        self._balance += amount
        
        # Проверяем, погашаем ли мы кредит
        credit_repaid = False
        if old_balance < 0:
            credit_repaid = True
        
        # Запись об успешной операции
        good_record = {
            'type': 'deposit',
            'amount': amount,
            'timestamp': datetime.datetime.now(),
            'balance_after': self._balance,
            'status': 'success',
            'description': f'Пополнение на {amount}. Баланс был {old_balance}, стал {self._balance}',
            'credit_used': False,
            'credit_repaid': credit_repaid,
            'available_credit': self._balance + self.credit_limit
        }
        
        # Добавляем информацию о погашении кредита если было
        if credit_repaid:
            repaid_amount = min(amount, abs(old_balance))
            good_record['description'] += f'. Погашено кредита: {repaid_amount}'
        
        self.operations_history.append(good_record)
        return True
    
    # Метод для показа информации о кредитном счете
    def show_info(self):
        print(f"Владелец КРЕДИТНОГО счета: {self.holder}")
        print(f"Текущий баланс: {self._balance}")
        print(f"Кредитный лимит: {self.credit_limit}")
        print(f"Доступный кредит: {self.get_available_credit()}")
        print(f"Всего операций: {len(self.operations_history)}")


# Пример использования - как будто студент тестирует свою программу
if __name__ == "__main__":
    print("=== Тест обычного счета ===")
    print()
    
    # Создаем обычный счет
    my_account = Account("Иван Петров", 1000)
    my_account.show_info()
    print()
    
    # Делаем операции
    print("Пополняем на 500...")
    my_account.deposit(500)
    print(f"Баланс: {my_account.get_balance()}")
    print()
    
    print("Снимаем 300...")
    my_account.withdraw(300)
    print(f"Баланс: {my_account.get_balance()}")
    print()
    
    print("Пытаемся снять 2000 (должно не получиться)...")
    my_account.withdraw(2000)
    print(f"Баланс: {my_account.get_balance()}")
    print()
    
    print("Пытаемся пополнить на -100 (должно не получиться)...")
    my_account.deposit(-100)
    print(f"Баланс: {my_account.get_balance()}")
    print()
    
    print("=== История операций обычного счета ===")
    history = my_account.get_history()
    for i, record in enumerate(history, 1):
        print(f"{i}. {record['timestamp'].strftime('%Y-%m-%d %H:%M:%S')} - {record['description']} ({record['status']})")
    
    print()
    print("="*50)
    print()
    
    print("=== Тест кредитного счета ===")
    print()
    
    # Создаем кредитный счет
    my_credit = CreditAccount("Петр Иванов", credit_limit=5000, balance=1000)
    my_credit.show_info()
    print()
    
    print("Снимаем 3000...")
    my_credit.withdraw(3000)
    print(f"Баланс: {my_credit.get_balance()}, Доступный кредит: {my_credit.get_available_credit()}")
    print()
    
    print("Снимаем еще 4000 (часть за счет кредита)...")
    my_credit.withdraw(4000)
    print(f"Баланс: {my_credit.get_balance()}, Доступный кредит: {my_credit.get_available_credit()}")
    print()
    
    print("Пополняем на 2000 (часть пойдет на погашение кредита)...")
    my_credit.deposit(2000)
    print(f"Баланс: {my_credit.get_balance()}, Доступный кредит: {my_credit.get_available_credit()}")
    print()
    
    print("Пытаемся снять 10000 (должно не получиться - превышение лимита)...")
    my_credit.withdraw(10000)
    print(f"Баланс: {my_credit.get_balance()}, Доступный кредит: {my_credit.get_available_credit()}")
    print()
    
    print("=== История операций кредитного счета ===")
    credit_history = my_credit.get_history()
    for i, record in enumerate(credit_history, 1):
        print(f"{i}. {record['timestamp'].strftime('%Y-%m-%d %H:%M:%S')} - {record['description']} ({record['status']})")

=== Тест обычного счета ===

Владелец счета: Иван Петров
Текущий баланс: 1000
Всего операций: 1

Пополняем на 500...
Баланс: 1500

Снимаем 300...
Баланс: 1200

Пытаемся снять 2000 (должно не получиться)...
Баланс: 1200

Пытаемся пополнить на -100 (должно не получиться)...
Баланс: 1200

=== История операций обычного счета ===
1. 2025-12-27 20:28:49 - Открытие счета. Баланс: 1000 (success)
2. 2025-12-27 20:28:49 - Пополнение на 500. Баланс был 1000, стал 1500 (success)
3. 2025-12-27 20:28:49 - Снятие 300. Баланс был 1500, стал 1200 (success)
4. 2025-12-27 20:28:49 - Ошибка: недостаточно средств. Хотите снять 2000, а на счете только 1200 (fail)
5. 2025-12-27 20:28:49 - Ошибка: сумма -100 должна быть больше 0 (fail)


=== Тест кредитного счета ===

Владелец КРЕДИТНОГО счета: Петр Иванов
Текущий баланс: 1000
Кредитный лимит: 5000
Доступный кредит: 5000
Всего операций: 1

Снимаем 3000...
Баланс: -2000, Доступный кредит: 3000

Снимаем еще 4000 (часть за счет кредита)...
Баланс: -2000, Доступн